# COMP0173: Coursework 2

The paper HEARTS: A Holistic Framework for Explainable, Sustainable, and Robust Text Stereotype Detection by Theo King, Zekun Wu et al. (2024) presents a comprehensive approach to analysing and detecting stereotypes in text [1]. The authors introduce the HEARTS framework, which integrates model explainability, carbon-efficient training, and accurate evaluation across multiple bias-sensitive datasets. By using transformer-based models such as ALBERT-V2, BERT, and DistilBERT, this research project demonstrates that stereotype detection performance varies significantly across dataset sources, underlining the need for diverse evaluation benchmarks. The paper provides publicly available datasets and code [2], allowing full reproducibility and offering a standardised methodology for future research on bias and stereotype detection in Natural Language Processing (NLP).

While the HEARTS framework evaluates stereotype detection in English, this project adapts the methodology to the Russian context. Russian stereotypes often rely on grammatical gender, morphology, and culture specific tropes. Although Russian is not classified as a low-resource language and many high-performing NLP models are available, there is currently no publicly accessible model specifically designed to detect stereotypes in Russian language. Existing models detecting toxicity or sentiment identify stereotypical and biased sentences only when they include specific patterns, such as insults, slurs, or identity-specific hate speech [8]. 

To address this gap, I introduce two fine-tuned classifiers, `AI-Forever-RuBert` [10] and `XML-RoBERTa` [11] trained on datasets `RBSA`, and `RBS`, respectively. Understanding these patterns is essential for applications such as content moderation, ensuring the safety of Russian-language LLMs, and monitoring harmful narratives across demographic groups and underrepresented societies. Adapting the HEARTS framework to this new sociolinguistic context illustrates its transferability beyond the English-speaking context and enables a more culturally grounded approach to bias detection, thereby promoting SDG 5: Gender Equality, SDG 10: Reduced Inequalities, and SDG 16: Peace, Justice, and Strong Institutions [5].

# Instructions

All figures produced during this notebook are stored in the project’s `COMP0173_Figures` directory.
The corresponding LaTeX-formatted performance comparison tables, jupyter notebooks are stored in `/COMP0173_PDF`. 
The compiled document are available as `COMP0173-CW2-TABLES.pdf` and `COMP0173_PDF/COMP0173-CW2-NOTEBOOK-XX.pdf`.
All prompts used for data augmentation are stored in `COMP0173_Prompts` and the manually collected stereotypes (with English translations) are provided in `COMP0173_Stereotypes`. 
The datasets used for model training and evaluation are stored in `COMP0173_Data` which contains: 

- rubias.tsv — RuBias dataset [6, 7]
- ruster.csv — RuSter dataset (see Part 2 of the notebook for source websites)
- rubist.csv — RBS dataset: RuBias + RuSter augmented with LLM-generated samples (Claude Sonnet), using a zero-shot prompt with examples
- rubist_second.csv — RBSA dataset: RuBias + RuSter augmented with LLM-generated samples using a second prompt version without examples

The notebooks `COMP0173_PDF/COMP0173-CW2-NOTEBOOK-P3.pdf` and `COMP0173_PDF/COMP0173-CW2-NOTEBOOK-P5.pdf` are replications of `COMP0173_PDF/COMP0173-CW2-NOTEBOOK-P2.pdf` and `COMP0173_PDF/COMP0173-CW2-NOTEBOOK-P4.pdf`, where P2 provides the new `RBSA` with second prompt (without examples) and P5 demonstrates the model running ON GPU (the results saved are from GPU fine-tuning).

# Technical Implementation (70%)

In [ ]:
# Import libraries 
import random, numpy as np, torch
import pandas as pd

import platform
import transformers
from datasets import load_dataset
import spacy 
import os
import sys
import importlib.util, pathlib
from pathlib import Path
import warnings 
from importlib import reload
from importlib.machinery import SourceFileLoader
from IPython.display import display

In [ ]:
# Check the GPU host (UCL access)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

CUDA available: True
Device: NVIDIA GeForce RTX 3090 Ti


In [ ]:
# Path
import os
os.chdir("/tmp/HEARTS-Text-Stereotype-Detection")
os.getcwd()

'/tmp/HEARTS-Text-Stereotype-Detection'

## Part 1: Replicate the baseline AI methodology using the open dataset 

The HEARTS framework evaluates stereotype detection using four publicly available text datasets [3]. The following datasets are Multi-Grain Stereotype Dataset (MGSD), Augmented WinoQueer (AWinoQueer), Augmented SeeGULL (ASeeGULL), and Expanded Multi-Grain Stereotype Dataset (EMGSD), which includes labelled stereotypical and non-stereotypical statements covering gender, profession, nationality, race, religion, and LGBTQ+ stereotypes. All datasets referenced in the paper are openly accessible through the associated GitHub repository [2] and Kaggle [3]. 

### Helper Functions

In [ ]:
def collect_summary(project_root: Path) -> pd.DataFrame:
    
    """
    Collect summary metrics from all model output folders.

    Parameters
    ----------
    project_root : Path
        Path to the root directory containing result_output_* folders.

    Returns
    -------
    pd.DataFrame
        A table containing model type, training dataset, evaluation dataset,
        accuracy, macro precision, macro recall, and macro F1 score.
    """
    
    rows = []

    # Loop through each model type directory
    for model_dir in [
        "result_output_albertv2",
        "result_output_bert",
        "result_output_distilbert",
    ]:
        base = project_root / model_dir
        if not base.exists():
            continue

        # Standardise model name
        model_name = model_dir.replace("result_output_", "")

        # Iterate through training subsets
        for train_dir in sorted(base.iterdir()):
            if not train_dir.is_dir():
                continue
            train_name = train_dir.name

            # Iterate through evaluation subsets
            for eval_dir in sorted(train_dir.iterdir()):
                if not eval_dir.is_dir():
                    continue
                eval_name = eval_dir.name

                report_path = eval_dir / "classification_report.csv"
                if not report_path.exists():
                    continue

                # Load sklearn classification_report CSV
                rep = pd.read_csv(report_path, index_col=0)

                # Extract accuracy and macro-level metrics
                acc = rep.loc["accuracy", "precision"]  # sklearn stores accuracy here
                macro_f1 = rep.loc["macro avg", "f1-score"]
                macro_prec = rep.loc["macro avg", "precision"]
                macro_rec = rep.loc["macro avg", "recall"]

                rows.append(
                    {
                        "model": model_name,
                        "train_on": train_name,
                        "eval_on": eval_name,
                        "accuracy": acc,
                        "macro_precision": macro_prec,
                        "macro_recall": macro_rec,
                        "macro_f1": macro_f1,
                    }
                )

    return pd.DataFrame(rows)

In [ ]:
def build_table(summary_df: pd.DataFrame) -> pd.DataFrame:
    
    """
    Construct a pivot table matching the layout of the HEARTS paper.

    Parameters
    ----------
    summary_df : pd.DataFrame
        DataFrame containing model, training set, evaluation set, and metrics.

    Returns
    -------
    pd.DataFrame
        Pivot table indexed by (model, training dataset short name) with
        evaluation datasets as columns and macro-F1 scores as values.
    """

    # Add short labels using mapping dictionaries
    df = summary_df.assign(
        train_short=lambda d: d["train_on"].map(train_map),
        eval_short=lambda d: d["eval_on"].map(eval_map),
    )

    # Remove rows where mapping failed
    df = df.dropna(subset=["train_short", "eval_short"])

    # Create pivot table: rows = model x train_set, cols = eval_set
    table = df.pivot_table(
        index=["model", "train_short"], columns="eval_short", values="macro_f1"
    ).sort_index()

    return table

### $\color{pink}{Question\ 1:}$ Clone the original repository successfully 

The code for HEARTS was obtained by cloning a fork of the official repository [2]. The upstream repository, authored by Holistic AI, was then linked to provide access to the original implementation. 

In [2]:
# # Clone fork
# git clone https://github.com/n1nt3nd0sw1tch/HEARTS-Text-Stereotype-Detection.git
# cd HEARTS-Text-Stereotype-Detection

# # Link to the original repo
# git remote add upstream https://github.com/holistic-ai/HEARTS-Text-Stereotype-Detection.git
# git remote -v   

### $\color{pink}{Question\ 2:}$ Document all dependencies and environment setup

A virtual environment was installed to isolate the HEARTS experimentation setup and ensure reproducibility. In the GPU-powered environment, the repository and data paths, along with system and library versions, were set up and confirmed [4]. All four datasets required by the HEARTS pipeline were confirmed to be present in the configured data directory.

**Runtime configuration on the UCL GPU node:**

- Working directory: /tmp/HEARTS-Text-Stereotype-Detection

- Python: 3.9.21

- Platform: Linux (x86_64, glibc 2.34)

- PyTorch: 2.8.0+cu128

- CUDA: available (NVIDIA GeForce RTX 3090 Ti) / (NVIDIA Tesla 4)

- Transformers: 4.57.3

In [ ]:
# # Create and activate conda environment
# conda create -n hearts python=3.10 -y
# conda activate hearts

# # Register environment as a Jupyter kernel
# python -m pip install ipykernel
# python -m ipykernel install --user

# # Install repository dependencies
# pip install -r requirements.txt
# pip install --no-cache-dir -r requirements.txt
# pip install --no-cache-dir spacy
# pip install --no-cache-dir scikit-learn
# pip install codecarbon
# python -m spacy download en_core_web_lg

In [ ]:
# accelerate==1.1.1
# aiohappyeyeballs==2.4.3
# aiohttp==3.11.8
# aiosignal==1.3.1
# annotated-types==0.7.0
# anyio==4.6.2.post1
# appnope==0.1.4
# arrow==1.3.0
# asttokens==3.0.1
# async-timeout==5.0.1
# attrs==24.2.0
# beautifulsoup4==4.14.3
# blis==1.0.1
# Bottleneck==1.4.2
# catalogue==2.0.10
# certifi==2024.8.30
# cffi==1.17.1
# charset-normalizer==3.4.0
# click==8.1.7
# cloudpathlib==0.16.0
# cloudpickle==3.1.0
# codecarbon==2.8.0
# colorama==0.4.6
# comm==0.2.3
# confection==0.1.5
# contourpy==1.3.1
# cryptography==44.0.0
# cycler==0.12.1
# cymem==2.0.10
# datasets==3.1.0
# DAWG-Python==0.7.2
# DAWG2-Python==0.9.0
# debugpy==1.8.16
# decorator==5.2.1
# deep-translator==1.11.4
# dill==0.3.8
# distro==1.9.0
# docopt==0.6.2
# dotenv==0.9.9
# en_core_web_lg==3.8.0
# et_xmlfile==2.0.0
# exceptiongroup==1.3.1
# executing==2.2.1
# fief-client==0.20.0
# filelock==3.16.1
# fonttools==4.55.0
# frozenlist==1.5.0
# fsspec==2024.9.0
# h11==0.14.0
# hf-xet==1.2.0
# httpcore==1.0.7
# httpx==0.27.2
# huggingface-hub==0.26.3
# idna==3.10
# imageio==2.36.1
# importlib_metadata==8.7.0
# intervaltree==3.1.0
# ipykernel==7.1.0
# ipymarkup==0.9.0
# ipython==8.37.0
# jedi==0.19.2
# Jinja2==3.1.4
# jiter==0.12.0
# joblib==1.4.2
# jupyter_client==8.6.3
# jupyter_core==5.9.1
# jwcrypto==1.5.6
# kiwisolver==1.4.7
# langcodes==3.5.0
# language_data==1.3.0
# lazy_loader==0.4
# lime==0.2.0.1
# llvmlite==0.43.0
# logging==0.4.9.6
# marisa-trie==1.2.1
# markdown-it-py==3.0.0
# MarkupSafe==3.0.2
# matplotlib==3.9.2
# matplotlib-inline==0.2.1
# mdurl==0.1.2
# mpmath==1.3.0
# multidict==6.1.0
# multiprocess==0.70.16
# murmurhash==1.0.11
# natasha==1.6.0
# navec==0.10.0
# nest_asyncio==1.6.0
# networkx==3.4.2
# nltk==3.9.2
# numba==0.60.0
# numexpr==2.14.1
# numpy==2.0.2
# openai==2.9.0
# openpyxl==3.1.5
# packaging==24.2
# pandas==2.3.3
# parso==0.8.5
# pexpect==4.9.0
# pickleshare==0.7.5
# pillow==11.0.0
# pip==25.3
# platformdirs==4.5.1
# preshed==3.0.9
# prometheus_client==0.21.0
# prompt-toolkit==3.0.36
# propcache==0.2.0
# psutil==6.1.0
# ptyprocess==0.7.0
# pure_eval==0.2.3
# py-cpuinfo==9.0.0
# pyarrow==18.1.0
# pycparser==2.22
# pydantic==2.10.2
# pydantic_core==2.27.1
# Pygments==2.18.0
# pymorphy2==0.9.1
# pymorphy2-dicts-ru==2.4.417127.4579844
# pymorphy3==2.0.6
# pymorphy3-dicts-ru==2.4.417150.4580142
# pynvml==11.5.3
# pyparsing==3.2.0
# python-dateutil==2.9.0.post0
# python-dotenv==1.2.1
# pytz==2024.2
# PyYAML==6.0.2
# pyzmq==27.1.0
# questionary==2.0.1
# RapidFuzz==3.10.1
# razdel==0.5.0
# regex==2024.11.6
# requests==2.32.3
# rich==13.9.4
# ru_core_news_lg==3.8.0
# russian-paraphrasers==0.0.3
# safetensors==0.4.5
# scikit-image==0.24.0
# scikit-learn==1.6.0rc1
# scipy==1.14.1
# seaborn==0.13.2
# sentence-transformers==0.4.0
# sentencepiece==0.2.1
# setuptools==75.6.0
# shap==0.46.0
# shellingham==1.5.4
# six==1.16.0
# slicer==0.0.8
# slovnet==0.6.0
# smart-open==6.4.0
# sniffio==1.3.1
# sortedcontainers==2.4.0
# soupsieve==2.8
# spacy==3.8.2
# spacy-legacy==3.0.12
# spacy-loggers==1.0.5
# srsly==2.4.8
# stack_data==0.6.3
# sympy==1.13.1
# termcolor==2.3.0
# thinc==8.3.2
# threadpoolctl==3.5.0
# tifffile==2024.9.20
# tokenizers==0.20.3
# torch==2.5.1
# torchvision==0.20.1
# tornado==6.5.1
# tqdm==4.67.1
# traitlets==5.14.3
# transformers==4.46.3
# typer==0.9.4
# types-python-dateutil==2.9.0.20241003
# typing_extensions==4.12.2
# tzdata==2024.2
# urllib3==2.2.3
# wasabi==1.1.3
# wcwidth==0.2.13
# weasel==0.3.4
# wheel==0.45.1
# wordcloud==1.9.4
# xxhash==3.5.0
# yargy==0.16.0
# yarl==1.18.0
# yaspin==3.1.0
# zipp==3.23.0

In [ ]:
# Path
REPO_DIR = Path("/tmp/HEARTS-Text-Stereotype-Detection").resolve()
DATA_DIR = REPO_DIR / "Model Training and Evaluation"

# Change working directory to the repo
os.chdir(REPO_DIR)

In [23]:
print("Current working dir:", Path.cwd())
print("Repository directory:", REPO_DIR)
print("Data directory:", DATA_DIR)

print("Python version:", sys.version)
print("Platform:", platform.platform())

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
print("Transformers version:", transformers.__version__)

Current working dir: /tmp/HEARTS-Text-Stereotype-Detection
Repository directory: /tmp/HEARTS-Text-Stereotype-Detection
Data directory: /tmp/HEARTS-Text-Stereotype-Detection/Model Training and Evaluation
Python version: 3.9.21 (main, Aug 19 2025, 00:00:00) 
[GCC 11.5.0 20240719 (Red Hat 11.5.0-5)]
Platform: Linux-5.14.0-570.58.1.el9_6.x86_64-x86_64-with-glibc2.34
PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3090 Ti
Transformers version: 4.57.3


In [24]:
# Check the csv files
csv_files = [
    "MGSD.csv",
    "Winoqueer - GPT Augmentation.csv",
    "SeeGULL - GPT Augmentation.csv",

    # MGSD Expanded is located elsewhere
    "../Exploratory Data Analysis/MGSD - Expanded.csv",
]

print("\nChecking dataset files:\n")
for f in csv_files:
    path = (DATA_DIR / f).resolve()
    print(f"{f:45} -> {'FOUND' if path.exists() else 'MISSING'}")


Checking dataset files:

MGSD.csv                                      -> FOUND
Winoqueer - GPT Augmentation.csv              -> FOUND
SeeGULL - GPT Augmentation.csv                -> FOUND
../Exploratory Data Analysis/MGSD - Expanded.csv -> FOUND


### $\color{pink}{Question\ 3:}$ Reproduce baseline results within ±5% of original paper metrics 

To maintain consistency with the HEARTS baseline, all experiments were conducted using a fixed random seed across Python, NumPy, and PyTorch (both CPU and CUDA). A dedicated module `BERT_Models_Fine_Tuning_Replication.py` was duplicated and used from the original `BERT_Models_Fine_Tuning.py`, with minimal fixes to file paths, training arguments, and compatibility issues, and a single entry-point function.

The `run_full_hearts_pipeline()` function:

1. Trains ALBERT, BERT, and DistilBERT on each of the four training settings
(mgsd_trained, winoqueer_gpt_augmentation_trained,
seegull_gpt_augmentation_trained,
merged_winoqueer_seegull_gpt_augmentation_trained), and

2. Evaluates each model on the four evaluation sets
(mgsd, winoqueer_gpt_augmentation,
seegull_gpt_augmentation,
merged_winoqueer_seegull_gpt_augmentation).

For each configuration, the accuracy, macro-precision, macro-recall, and macro-F1 scores are recorded in a CSV file and summarised in a compact results table. The accuracy and macro-F1 scores achieved in this study for all three model families and most dataset combinations are within approximately $±5$ percentage points of the metrics reported in the HEARTS paper, thereby meeting the baseline replication requirement.

In [25]:
# Set a fixed seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Provide the path to the python file 
module_path = pathlib.Path("Model Training and Evaluation/BERT_Models_Fine_Tuning_Replication.py")

# Upload models 
spec = importlib.util.spec_from_file_location("hearts_models", module_path)
hearts_models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(hearts_models)

# Run
hearts_models.run_full_hearts_pipeline()

Loading: /tmp/HEARTS-Text-Stereotype-Detection/Model Training and Evaluation/MGSD.csv
First few examples from the training data:
        group                                               text  label  \
38518    race  Syria is a well known area nowadays. There is ...      1   
12979    race                   The people of germany are angry.      1   
35494    race  Lebanon is a small country in the Middle East ...      1   
42213    race  We have a french tutor. She is very humble and...      0   
16293  gender                              Every male acts dumb.      1   

      data_name  
38518      MGSD  
12979      MGSD  
35494      MGSD  
42213      MGSD  
16293      MGSD  
First few examples from the testing data:
            group                                               text  label  \
17851        race      When i see an arab, i think they are a happy.      0   
37449        race  People in India dress differently. The types o...      0   
23634      gender    Everyone has

loading configuration file config.json from cache at /cs/student/msc/aisd/2025/bekenova/.cache/huggingface/hub/models--albert--albert-base-v2/snapshots/8e2f239c5f8a2c0f253781ca60135db913e5c80c/config.json
Model config AlbertConfig {
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

loading

Sample tokenized input from train: {'group': 'religion', 'text': 'Have you ever met someone named Mohammed? I need a new pair of pajamas.', 'label': 0, 'data_name': 'MGSD', '__index_level_0__': 36953, 'input_ids': [2, 57, 42, 462, 798, 737, 377, 13479, 60, 31, 376, 21, 78, 2146, 16, 23358, 18, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


Map: 100%|████████████████████████| 8299/8299 [00:00<00:00, 25612.81 examples/s]
PyTorch: setting up devices
Model Training and Evaluation/BERT_Models_Fine_Tuning2.py:209: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Sample tokenized input from validation: {'group': 'race', 'text': 'My friend is German and sometimes we hand out. He gets very angry sometimes.', 'label': 1, 'data_name': 'MGSD', '__index_level_0__': 36651, 'input_ids': [2, 51, 860, 25, 548, 17, 1030, 95, 224, 70, 9, 24, 3049, 253, 3365, 1030, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 1}


...

Sample tokenized input from test: {'group': 'race', 'text': 'When i see an arab, i think they are a happy.', 'label': 0, 'data_name': 'MGSD', '__index_level_0__': 17851, 'input_ids': [101, 2043, 1045, 2156, 2019, 5424, 1010, 1045, 2228, 2027, 2024, 1037, 3407, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}


loading configuration file model_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/model.safetensors
loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.jso

Saved full results to: result_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/mgsd/full_results.csv
Saved classification report to: result_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/mgsd/classification_report.csv
Number of unique labels: 2


Map: 100%|██████████████████████| 11441/11441 [00:00<00:00, 30577.80 examples/s]
Device set to use cuda:0
/tmp/hearts_gpu/lib64/python3.9/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Sample tokenized input from test: {'text': 'The Somali refugees were labeled as unacquainted by some', 'label': 0, 'group': 'Somalis', 'data_name': 'SeeGULL - GPT Augmentation', 'input_ids': [101, 1996, 16831, 8711, 2020, 12599, 2004, 14477, 2278, 16211, 18447, 2098, 2011, 2070, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}
Saved full results to: result_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/merged_winoqueer_seegull_gpt_augmentation/full_results.csv
Saved classification report to: result_output_bert/merged_winoqueer_seegull_gpt_augmentation_trained/merged_winoqueer_seegull_gpt_augmentation/classification_report.csv


### $\color{pink}{Question\ 3.1:}$ Results 

The expected model checkpoints, tokenizer files, and configuration artefacts were successfully generated:

- /model_output_albertv2/mgsd_trained/

- /model_output_albertv2/winoqueer_gpt_augmentation_trained/

- /model_output_albertv2/seegull_gpt_augmentation_trained/

- /model_output_albertv2/merged_winoqueer_seegull_gpt_augmentation_trained/

- /result_output_albertv2

- /result_output_bert

- /result_output_distilbert

In [ ]:
# Check the outputs directory 
output_dir = "/tmp/HEARTS-Text-Stereotype-Detection/model_output_albertv2"
print(os.listdir(output_dir))

['mgsd_baseline', 'mgsd_trained', 'winoqueer_gpt_augmentation_trained', 'seegull_gpt_augmentation_trained', 'merged_winoqueer_seegull_gpt_augmentation_trained']


In [ ]:
# Check the outputs directory for MSGD trained model 
mgsd_path = "/tmp/HEARTS-Text-Stereotype-Detection/model_output_albertv2/mgsd_trained"
print(os.listdir(mgsd_path))

['checkpoint-1038', 'config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json', 'training_args.bin']


In [ ]:
# Define the project root
PROJECT_ROOT = Path("/tmp/HEARTS-Text-Stereotype-Detection")

# Build the summary table
summary_df = collect_summary(PROJECT_ROOT).sort_values(["model", "train_on", "eval_on"])
display(summary_df)

,model,train_on,eval_on,accuracy,macro_precision,macro_recall,macro_f1
0,albertv2,merged_winoqueer_seegull_gpt_augmentation_trained,merged_winoqueer_seegull_gpt_augmentation,0.829560,0.812057,0.804933,0.808239
1,albertv2,merged_winoqueer_seegull_gpt_augmentation_trained,mgsd,0.817525,0.798737,0.791267,0.794700
2,albertv2,merged_winoqueer_seegull_gpt_augmentation_trained,seegull_gpt_augmentation,0.898551,0.886902,0.884058,0.885452
3,albertv2,merged_winoqueer_seegull_gpt_augmentation_trained,winoqueer_gpt_augmentation,0.977029,0.973671,0.974750,0.974207
4,albertv2,mgsd_trained,merged_winoqueer_seegull_gpt_augmentation,0.806485,0.795266,0.760456,0.772482
5,albertv2,mgsd_trained,mgsd,0.813187,0.802741,0.769727,0.781473
6,albertv2,mgsd_trained,seegull_gpt_augmentation,0.748792,0.733152,0.759058,0.735743
7,albertv2,mgsd_trained,winoqueer_gpt_augmentation,0.736600,0.815491,0.611225,0.602798
8,albertv2,seegull_gpt_augmentation_trained,merged_winoqueer_seegull_gpt_augmentation,0.699851,0.662657,0.653821,0.657215
9,albertv2,seegull_gpt_augmentation_trained,mgsd,0.687199,0.647613,0.638152,0.641524


In [ ]:
# Mapping from full training directory names to short labels
train_map = {
    "mgsd_trained": "MGSD",
    "winoqueer_gpt_augmentation_trained": "AWinQ",
    "seegull_gpt_augmentation_trained": "ASeeGULL",
    "merged_winoqueer_seegull_gpt_augmentation_trained": "EMGSD",
}

# Mapping from full evaluation directory names to short labels
eval_map = {
    "mgsd": "MGSD",
    "winoqueer_gpt_augmentation": "AWinQ",
    "seegull_gpt_augmentation": "ASeeGULL",
    "merged_winoqueer_seegull_gpt_augmentation": "EMGSD",
}

# Build and display the table
table = build_table(summary_df)
table

eval_short              ASeeGULL     AWinQ     EMGSD      MGSD
model      train_short                                        
albertv2   ASeeGULL     0.878625  0.758341  0.657215  0.641524
           AWinQ        0.742479  0.979570  0.643667  0.615635
           EMGSD        0.885452  0.974207  0.808239  0.794700
           MGSD         0.735743  0.602798  0.772482  0.781473
bert       ASeeGULL     0.889947  0.823317  0.653108  0.631083
           AWinQ        0.726933  0.991441  0.651479  0.627560
           EMGSD        0.891304  0.975954  0.825447  0.813525
           MGSD         0.711166  0.783431  0.803933  0.808953
distilbert ASeeGULL     0.883363  0.868559  0.648040  0.621496
           AWinQ        0.701243  0.974379  0.648156  0.622647
           EMGSD        0.908891  0.984524  0.805193  0.790040
           MGSD         0.752678  0.753465  0.781091  0.783385

### Table: HEARTS Replicated Results

The table below summarises the replicated Macro F1 scores for ALBERT-V2, BERT, and DistilBERT across all combinations of training and evaluation datasets.

![HEARTS Replicated Results](COMP0173_Figures/hearts_replicated_results.png)

### Table: HEARTS Original Results [1]

![Model Results Table](COMP0173_Figures/hearts_results.png)

### Results Interpretation

The replicated Macro F1 scores for ALBERT-V2, BERT, and DistilBERT closely match the results reported in the HEARTS paper, with a few exceptions. For nearly all train-test combinations, the reproduced valuesexcept for two; fall within the required $±5\%$ range, confirming the successful replication of the baseline AI methodology.

#### ALBERT-V2

For ALBERT-V2, the replicated Macro F1 scores generally align with the original results. Most train-test combinations remain within a five percentage point margin, and the relative performance ordering across datasets is constant with the behaviour defined in the HEARTS paper. Only a couple of values diverge slightly beyond this margin, particularly those involving evaluations on `AWinoQueer`.

![ALBERT-V2 Results Table](COMP0173_Figures/results_comparison_albert.png)

#### BERT

The BERT model shows remarkable reproducibility. All replicated scores are consistently within a $±5\%$ range, with differences between the original and replicated values typically being minor, often just one or two points apart. 

![BERT Results Table](COMP0173_Figures/results_comparison_bert.png)

#### DistilBERT

DistilBERT demonstrates a strong alignment between the original and replicated metrics. Every train-test result falls within the target deviation range. 

![DistilBERT-V2 Results Table](COMP0173_Figures/results_comparison_distilbert.png)

# References 

[1] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. 2024.
HEARTS: A holistic framework for explainable, sustainable and robust text stereotype detection.
arXiv preprint arXiv:2409.11579.
Available at: https://arxiv.org/abs/2409.11579
(Accessed: 4 December 2025).
https://doi.org/10.48550/arXiv.2409.11579

[2] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. 2024.
HEARTS-Text-Stereotype-Detection (GitHub Repository).
Available at: https://github.com/holistic-ai/HEARTS-Text-Stereotype-Detection
(Accessed: 4 December 2025).

[3] Theo King, Zekun Wu, Adriano Koshiyama, Emre Kazim, and Philip Treleaven. Holistic AI. 2024.
EMGSD: Expanded Multi-Group Stereotype Dataset (HuggingFace Dataset).
Available at: https://huggingface.co/datasets/holistic-ai/EMGSD
(Accessed: 4 December 2025).

[4] University College London Technical Support Group (TSG).
2025. GPU Access and Usage Documentation.
Available at: https://tsg.cs.ucl.ac.uk/gpus/
(Accessed: 6 December 2025).

[5] United Nations. 2025. The 2030 Agenda for Sustainable Development. 
Available at: https://sdgs.un.org/2030agenda 
(Accessed: 6 December 2025).

[6] Veronika Grigoreva, Anastasiia Ivanova, Ilseyar Alimova, and Ekaterina Artemova. 2024.
RuBia: A Russian Language Bias Detection Dataset.
Available at: https://arxiv.org/abs/2403.17553
(Accessed: 9 December 2025).

[7] Veronika Grigoreva, Anastasiia Ivanova, Ilseyar Alimova, and Ekaterina Artemova. 2024.
RuBia-Dataset (GitHub Repository).
Available at: https://github.com/vergrig/RuBia-Dataset
(Accessed: 9 December 2025).

[8] Sismetanin. 2020. Toxic Comments Detection in Russian (GitHub Repository).
Available at: https://github.com/sismetanin/toxic-comments-detection-in-russian
(Accessed: 9 December 2025).

[9] DeepPavlov. 2019. RuBERT-base-cased (Hugging Face Model).
Available at: https://huggingface.co/DeepPavlov/rubert-base-cased
(Accessed: 9 December 2025).

[10] AI-Forever. 2023. RuBERT-base (Hugging Face Model).
Available at: https://huggingface.co/ai-forever/ruBert-base
(Accessed: 9 December 2025).

[11] Hugging Face. 2024. XLM-RoBERTa: Model Documentation.
Available at: https://huggingface.co/docs/transformers/en/model_doc/xlm-roberta
(Accessed: 9 December 2025).

[12] DeepPavlov. 2020. ruBERT-base-cased-sentence (Hugging Face Model).
Available at: https://huggingface.co/DeepPavlov/rubert-base-cased-sentence
(Accessed: 9 December 2025).